# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 2:43PM,240553,10,8200464,"Snap Medical Industries, LLC",Released
1,Jun 27 2022 2:43PM,240553,10,8200465,"Snap Medical Industries, LLC",Released
2,Jun 27 2022 2:43PM,240553,10,8200466,"Snap Medical Industries, LLC",Released
3,Jun 27 2022 2:43PM,240553,10,8200467,"Snap Medical Industries, LLC",Released
4,Jun 27 2022 2:43PM,240553,10,8200468,"Snap Medical Industries, LLC",Released
5,Jun 27 2022 2:43PM,240553,10,8200469,"Snap Medical Industries, LLC",Released
6,Jun 27 2022 2:43PM,240553,10,8200470,"Snap Medical Industries, LLC",Released
7,Jun 27 2022 2:43PM,240553,10,8200471,"Snap Medical Industries, LLC",Released
8,Jun 27 2022 2:43PM,240553,10,8200472,"Snap Medical Industries, LLC",Released
9,Jun 27 2022 2:43PM,240553,10,8200473,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,240549,Released,1
42,240550,Released,1
43,240551,Released,1
44,240552,Released,1
45,240553,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240549,NaN,NaN,1.0
240550,NaN,NaN,1.0
240551,NaN,NaN,1.0
240552,NaN,NaN,1.0
240553,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240465,0.0,0.0,1.0
240475,0.0,5.0,13.0
240476,0.0,1.0,0.0
240492,0.0,1.0,0.0
240495,12.0,2.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240465,0,0,1
240475,0,5,13
240476,0,1,0
240492,0,1,0
240495,12,2,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240465,0,0,1
1,240475,0,5,13
2,240476,0,1,0
3,240492,0,1,0
4,240495,12,2,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240465,,,1
1,240475,,5,13
2,240476,,1,
3,240492,,1,
4,240495,12,2,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 2:43PM,240553,10,"Snap Medical Industries, LLC"
14,Jun 27 2022 2:39PM,240552,10,Emerginnova
15,Jun 27 2022 2:38PM,240551,20,Sartorius Stedim North America
16,Jun 27 2022 2:14PM,240465,18,Innova Softgel LLC
17,Jun 27 2022 2:11PM,240550,17,Twinlab Consolidated Corporation
18,Jun 27 2022 1:58PM,240549,19,ACG North America LLC
19,Jun 27 2022 1:37PM,240548,16,Sartorius Lab Products and Service
21,Jun 27 2022 1:21PM,240545,20,Sartorius Stedim North America
22,Jun 27 2022 1:15PM,240544,20,"ACI Healthcare USA, Inc."
25,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 27 2022 2:43PM,240553,10,"Snap Medical Industries, LLC",,,14
1,Jun 27 2022 2:39PM,240552,10,Emerginnova,,,1
2,Jun 27 2022 2:38PM,240551,20,Sartorius Stedim North America,,,1
3,Jun 27 2022 2:14PM,240465,18,Innova Softgel LLC,,,1
4,Jun 27 2022 2:11PM,240550,17,Twinlab Consolidated Corporation,,,1
5,Jun 27 2022 1:58PM,240549,19,ACG North America LLC,,,1
6,Jun 27 2022 1:37PM,240548,16,Sartorius Lab Products and Service,,,2
7,Jun 27 2022 1:21PM,240545,20,Sartorius Stedim North America,,,1
8,Jun 27 2022 1:15PM,240544,20,"ACI Healthcare USA, Inc.",,,3
9,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 2:43PM,240553,10,"Snap Medical Industries, LLC",14,,
1,Jun 27 2022 2:39PM,240552,10,Emerginnova,1,,
2,Jun 27 2022 2:38PM,240551,20,Sartorius Stedim North America,1,,
3,Jun 27 2022 2:14PM,240465,18,Innova Softgel LLC,1,,
4,Jun 27 2022 2:11PM,240550,17,Twinlab Consolidated Corporation,1,,
5,Jun 27 2022 1:58PM,240549,19,ACG North America LLC,1,,
6,Jun 27 2022 1:37PM,240548,16,Sartorius Lab Products and Service,2,,
7,Jun 27 2022 1:21PM,240545,20,Sartorius Stedim North America,1,,
8,Jun 27 2022 1:15PM,240544,20,"ACI Healthcare USA, Inc.",3,,
9,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 2:43PM,240553,10,"Snap Medical Industries, LLC",14,,
1,Jun 27 2022 2:39PM,240552,10,Emerginnova,1,,
2,Jun 27 2022 2:38PM,240551,20,Sartorius Stedim North America,1,,
3,Jun 27 2022 2:14PM,240465,18,Innova Softgel LLC,1,,
4,Jun 27 2022 2:11PM,240550,17,Twinlab Consolidated Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 2:43PM,240553,10,"Snap Medical Industries, LLC",14.0,NaN,NaN
1,Jun 27 2022 2:39PM,240552,10,Emerginnova,1.0,NaN,NaN
2,Jun 27 2022 2:38PM,240551,20,Sartorius Stedim North America,1.0,NaN,NaN
3,Jun 27 2022 2:14PM,240465,18,Innova Softgel LLC,1.0,NaN,NaN
4,Jun 27 2022 2:11PM,240550,17,Twinlab Consolidated Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 2:43PM,240553,10,"Snap Medical Industries, LLC",14.0,0.0,0.0
1,Jun 27 2022 2:39PM,240552,10,Emerginnova,1.0,0.0,0.0
2,Jun 27 2022 2:38PM,240551,20,Sartorius Stedim North America,1.0,0.0,0.0
3,Jun 27 2022 2:14PM,240465,18,Innova Softgel LLC,1.0,0.0,0.0
4,Jun 27 2022 2:11PM,240550,17,Twinlab Consolidated Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2405225,53.0,27.0,17.0
12,240505,1.0,1.0,0.0
15,240503,5.0,0.0,15.0
16,721547,3.0,1.0,0.0
17,240550,1.0,0.0,0.0
18,240465,1.0,0.0,0.0
19,481075,2.0,23.0,0.0
20,2164784,70.0,40.0,0.0
21,1443062,4.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2405225,53.0,27.0,17.0
1,12,240505,1.0,1.0,0.0
2,15,240503,5.0,0.0,15.0
3,16,721547,3.0,1.0,0.0
4,17,240550,1.0,0.0,0.0
5,18,240465,1.0,0.0,0.0
6,19,481075,2.0,23.0,0.0
7,20,2164784,70.0,40.0,0.0
8,21,1443062,4.0,2.0,0.0
9,22,481056,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,53.0,27.0,17.0
1,12,1.0,1.0,0.0
2,15,5.0,0.0,15.0
3,16,3.0,1.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,19,2.0,23.0,0.0
7,20,70.0,40.0,0.0
8,21,4.0,2.0,0.0
9,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,12,Released,1.0
2,15,Released,5.0
3,16,Released,3.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,17.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,27.0,1.0,0.0,1.0,0.0,0.0,23.0,40.0,2.0,0.0
Released,53.0,1.0,5.0,3.0,1.0,1.0,2.0,70.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,17.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,27.0,1.0,0.0,1.0,0.0,0.0,23.0,40.0,2.0,0.0
2,Released,53.0,1.0,5.0,3.0,1.0,1.0,2.0,70.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,17.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,27.0,1.0,0.0,1.0,0.0,0.0,23.0,40.0,2.0,0.0
2,Released,53.0,1.0,5.0,3.0,1.0,1.0,2.0,70.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()